oid获取， F12——>Network——>XHR——>搜索oid——>archive?oid=

In [30]:
# 发送请求
import requests
import xlwt
# 每次请求停1s，太快会被B站拦截。
import time
import csv




# 爬虫类（面向对象）
class JsonProcess:
    def __init__(self):
        self.Json_data = ''
        # 请求头
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36 Edg/99.0.1150.39'
        }
	
    # 发送爬取请求
    def spider(self, URL):
        url = URL
        response = requests.get(url, headers=self.headers, verify=False)
        response.encoding = 'utf-8'
        self.Json_data = response.json()['data']['replies']


# 爬取子评论
def getSecondReplies(root, oid, name):
    reply = JsonProcess()
    # 页数
    pn = 1
    # 不知道具体有多少页的评论，所以使用死循环一直爬
    while True:
        url = f'https://api.bilibili.com/x/v2/reply/reply?jsonp=jsonp&pn={pn}&type=1&oid={oid}&ps=10&root={root}&_=1647581648753'
        # 没爬一次就睡1秒
        time.sleep(1)
        reply.spider(url)
        # 如果当前页为空（爬到头了），跳出子评论
        if reply.Json_data is None:
            break
        # 组装数据，存入数据
        for node in reply.Json_data:
            l = []
            ctime = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(node['ctime']))
            rpid = node['rpid']
            rname = node['member']['uname']
            avatar = node['member']['avatar']
            content = node['content']['message']
            like = node['like']
            rcount = node['rcount']
            action = node['action']
            mid = node['mid']
            sign = node['member']['sign']
            sex = node['member']['sex']
            l.append(ctime)
            l.append(content)
            l.append(None)
            l.append(rcount)
            l.append(action)
            l.append(mid)
            l.append(rname)
            l.append(sign)
            l.append(sex)
            l.append(None)
            l.append(None)
            l.append(name)
            writer.writerow(l)
        # 每爬完一次，页数加1
        pn += 1

        
# 爬取根评论
def getReplies(jp, i, oid):
    # 不知道具体有多少页的评论，所以使用死循环一直爬
    while True:
        url = f'https://api.bilibili.com/x/v2/reply/main?jsonp=jsonp&next={i}&type=1&oid={oid}&mode=3&plat=1&_=1647577851745'
        jp.spider(url)
        # 如果当前页为空（爬到头了），跳出循环，程序结束。
        if jp.Json_data is None:
            break
        # 组装数据，存入数据
        for node in jp.Json_data:
            l = []
            ctime = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(node['ctime']))
            rpid = node['rpid']
            name = node['member']['uname']
            avatar = node['member']['avatar']
            content = node['content']['message']
            like = node['like']
            rcount = node['rcount']
            action = node['action']
            mid = node['mid']
            sign = node['member']['sign']
            sex = node['member']['sex']
            if sex == '女':
                sex = 0
            else:
                sex = 1
            l.append(ctime)
            l.append(content)
            l.append(None)
            l.append(rcount)
            l.append(action)
            l.append(mid)
            l.append(name)
            l.append(sign)
            l.append(sex)
            l.append(None)
            l.append(None)
            l.append(None)
            writer.writerow(l)
            # 如果有子评论，爬取子评论
            if node['replies'] is not None:
                getSecondReplies(rpid, oid, name)
        # 每爬完一页，页数加1
        i += 1


In [31]:
if __name__ == '__main__':
    oid = input()
    theme = input()
    csvfile = open('./data/'+ theme +'.csv', 'w', newline='', encoding='utf-16')
    writer = csv.writer(csvfile)
    title = ['转发created_at', '转发text', '转发reposts_count', '转发comments_count', '转发attitudes_count', '转发id', '转发screen_name', '转发description', '转发gender', '转发followers_count', '转发follow_count','转发source_name']
    writer.writerow(title)
    JP = JsonProcess()
    getReplies(JP, 1, oid)
    print('\n================存储完成================\n')
    csvfile.close()

e:\Anaconda\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.bilibili.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
e:\Anaconda\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.bilibili.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
e:\Anaconda\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.bilibili.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
e:\Anaconda\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverifi


================存储完成================



e:\Anaconda\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.bilibili.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
